## Recap: 

By using a list of 83 sectors and 204 industries we were able to get an accuracy of 50% and 44%. This was achieved after sepeerating the API call into two. 

## Next steps 

Our next step would be to try and condense these lists, which we hope will increase accuracy and decrese waiting time. 

### Condensing the lists

Our challange advisors gave us a list of industries that their vendor uses, we will try to use the API with those industries as iptions and see how the accuracy changes. 

We will do this by: 

1. Creating a map from all existing industries to the ones we want to use. 
2. Applying the map to the data
3. Feed the new list and companies to the LLM 
4. Comapre the results and see accuracy with this new list

## Preparing the data set

We filter by only having full time and US jobs, then sepesrate the JSON company profile into sector and indsutry and drop company profile and job ID features.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv("job_descriptions.csv")

In [3]:
feature_to_filter_type = "Work Type"
values_to_keep_type = ["Full-Time"]

df_filtered_type = df[df[feature_to_filter_type].isin(values_to_keep_type)]

In [4]:
feature_to_filter_country = "Country"
values_to_keep_country = ["USA"]

df_filtered_country = df[df[feature_to_filter_country].isin(values_to_keep_country)]

In [5]:
import json

def extract_sector_industry(row):
  try:
    profile = json.loads(row['Company Profile'])
    return profile.get('Sector'), profile.get('Industry')
  except:
    return None, None

df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')

/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_6920/3398901514.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')
/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_6920/3398901514.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')


In [6]:
df_filtered_country.drop(columns=['Company Profile'], inplace=True)
df_filtered_country.drop(columns=['Job Id']).describe()

/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_6920/2787395556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country.drop(columns=['Company Profile'], inplace=True)


,latitude,longitude,Company Size
count,7.431000e+03,7.431000e+03,7431.000000
mean,3.709020e+01,-9.571290e+01,73073.289194
std,2.934739e-12,7.446989e-12,35201.315418
min,3.709020e+01,-9.571290e+01,12650.000000
25%,3.709020e+01,-9.571290e+01,42393.500000
50%,3.709020e+01,-9.571290e+01,72502.000000
75%,3.709020e+01,-9.571290e+01,103686.500000
max,3.709020e+01,-9.571290e+01,134824.000000


In [7]:
df_filtered_country.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Sector,Industry
156,579575048679091,3 to 9 Years,MCA,$59K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,126334,...,Network Engineer,Wireless Network Engineer,USAJOBS,"Wireless Network Engineers design, implement, ...","{'Flexible Spending Accounts (FSAs), Relocatio...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Lendlease Group,Real Estate/Construction,Real Estate
270,288941837754490,4 to 15 Years,B.Tech,$57K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,35869,...,Landscape Architect,Urban Planner,The Muse,Develop and implement urban planning strategie...,"{'Employee Referral Programs, Financial Counse...",Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",United Technologies Corporation,Aerospace/Building Systems,Aerospace and Defense
1140,2802376880223274,0 to 9 Years,B.Tech,$63K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,80163,...,UX Researcher,UX Strategist,SimplyHired,"Develop user experience (UX) strategies, creat...","{'Transportation Benefits, Professional Develo...",User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,International Flavors & Fragrances,Flavors & Fragrances,Chemicals
1591,2393222411935908,5 to 11 Years,MCA,$55K-$124K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,79017,...,HR Coordinator,Benefits Coordinator,Internships.com,Benefits Coordinators administer employee bene...,"{'Transportation Benefits, Professional Develo...",Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",KeyCorp,Financial Services,Commercial Banks
1609,2402523179167443,0 to 10 Years,MCA,$65K-$91K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,65144,...,Landscape Architect,Environmental Designer,USAJOBS,"Design outdoor spaces, parks, and landscapes w...","{'Childcare Assistance, Paid Time Off (PTO), R...",Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,Exelon,Energy,Utilities: Gas and Electric


In [19]:
df_filtered_country.shape

(7431, 24)

In [20]:
df_filtered_country['Industry'].isnull().sum()

40

In [21]:
df_filtered_country['Sector'].isnull().sum()

40

# Dealing with nulls

We decide to remove null values in this case since there is only 40 of them

In [24]:
df_filtered_country = df_filtered_country.dropna(subset=['Sector', 'Industry'])

In [25]:
df.to_csv("filtered_country_work_type.csv", index=False)

# Mapping

Load the real industy sectors and map them to the current sectors. Then add a new collumn with these mapped sectors

In [26]:
real_salary_sectors = [
    "Aerospace",
    "Construction",
    "Consumer Goods",
    "Coorporate Services",
    "Defense",
    "Design",
    "Education",
    "Energy and Mining",
    "Engineering",
    "Entertainment",
    "Finance",
    "Hardware and Newtowrking",
    "Healthcare",
    "Manufacturing",
    "Media and Communications",
    "Nonprofit",
    "Public Administration",
    "Real Estate",
    "Recreation and Travel",
    "Retail",
    "Software and IT Services",
    "Telecommunications",
    "Transportation and Logistics",
    "Wellness and Fitness"
]

In [28]:
data_sectors = df_filtered_country["Sector"].unique()
data_sectors.sort()
print(data_sectors)

['Advertising & Marketing' 'Advertising and Marketing'
 'Aerospace & Defense' 'Aerospace and Defense'
 'Aerospace/Building Systems' 'Aerospace/Defense' 'Agriculture'
 'Agrochemicals' 'Airlines' 'Aluminum' 'Apparel' 'Apparel and Footwear'
 'Appliances' 'Appliances and Electronics' 'Asset Management' 'Automotive'
 'Automotive Parts' 'Automotive/Motorcycles' 'Aviation and Travel'
 'Banking' 'Banking/Financial Services' 'Beauty' 'Beauty/Cosmetics'
 'Beverage' 'Beverage/Alcohol' 'Beverages' 'Biotechnology'
 'Building Materials' 'Building Materials & Equipment' 'Business Services'
 'Cement & Aggregates' 'Chemicals' 'Chemicals and Energy'
 'Communication Equipment' 'Communication Services' 'Conglomerate'
 'Construction' 'Construction & Engineering'
 'Construction and Engineering' 'Construction/Building Materials'
 'Construction/Infrastructure' 'Consumer Discretionary' 'Consumer Goods'
 'Consumer Goods/Homecare' 'Data and Analytics' 'Diversified' 'E-commerce'
 'E-commerce/Online Marketplace' '

In [47]:
print(data_sectors.shape)

(204,)


Since there are so many different possible value we deicide to use an OLLAMA call to map each unique sector to a real salary sector 

In [31]:
import requests
url = "http://localhost:11434/api/chat"

In [33]:
def mapSectors(prompt):
    real_salary_sectors_str = ", ".join(real_salary_sectors)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": f"Take {prompt} and map it to the closest sector from the following list: {real_salary_sectors_str}. Respond with only the sector that you think is the closest match. Do not add anything else to you response."

            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    return response


In [34]:
print(mapSectors("Advertising & Marketing"))

Media and Communications


In [35]:
mapped_sectors = []

for sector in data_sectors:
    new_sector = mapSectors(sector)
    mapped_sectors.append(new_sector)
    print(f"{sector} -> {new_sector}")

Advertising & Marketing -> Media and Communications
Advertising and Marketing -> Media and Communications
Aerospace & Defense -> Manufacturing
Aerospace and Defense -> Engineering
Aerospace/Building Systems -> Building Systems -> Construction
Aerospace/Defense -> Engineering
Agriculture -> Manufacturing
Agrochemicals -> Manufacturing
Airlines -> Transportation and Logistics
Aluminum -> Aerospace
Apparel -> Consumer Goods
Apparel and Footwear -> Consumer Goods
Appliances -> Manufacturing
Appliances and Electronics -> Consumer Goods
Asset Management -> Finance
Automotive -> Transportation and Logistics
Automotive Parts -> Manufacturing
Automotive/Motorcycles -> Transportation and Logistics
Aviation and Travel -> Transportation and Logistics
Banking -> Finance
Banking/Financial Services -> Finance
Beauty -> Design
Beauty/Cosmetics -> Consumer Goods
Beverage -> Consumer Goods
Beverage/Alcohol -> Consumer Goods
Beverages -> Beverages -> Consumer Goods
Biotechnology -> Healthcare
Building Ma

In [36]:
print(mapped_sectors)

['Media and Communications', 'Media and Communications', 'Manufacturing', 'Engineering', 'Building Systems -> Construction', 'Engineering', 'Manufacturing', 'Manufacturing', 'Transportation and Logistics', 'Aerospace', 'Consumer Goods', 'Consumer Goods', 'Manufacturing', 'Consumer Goods', 'Finance', 'Transportation and Logistics', 'Manufacturing', 'Transportation and Logistics', 'Transportation and Logistics', 'Finance', 'Finance', 'Design', 'Consumer Goods', 'Consumer Goods', 'Consumer Goods', 'Beverages -> Consumer Goods', 'Healthcare', 'Construction', 'Construction', 'Software and IT Services', 'Manufacturing', 'Manufacturing', 'Energy and Mining', 'Media and Communications', 'Media and Communications', 'Finance', 'Engineering', 'Engineering', 'Engineering', 'Manufacturing', 'Construction', 'Consumer Discretionary -> Entertainment', 'Manufacturing', 'Consumer Goods/Homecare → Consumer Goods', 'Software and IT Services', 'Software and IT Services', 'Consumer Goods', 'Retail', 'Softwa

In [44]:
# Map original sectors to mapped sectors
sector_mapping = dict(zip(data_sectors, mapped_sectors))
# Print so its easier to read 
for key, value in sector_mapping.items():
    # print only the ones where the value is not in real_salary_sectors
    if value not in real_salary_sectors:
        # Put these sectors through the API again 
        new_sector = mapSectors(key)
        value = new_sector
        print(f"{key} -> {value}")

Aerospace/Building Systems -> Construction
Beverages -> Consumer Goods
Consumer Discretionary -> Entertainment
Consumer Goods/Homecare -> Consumer Goods/Homecare: Consumer Goods
Electronics -> Hardware and Networking
Energy -> Energy and Mining
Energy/Infrastructure -> Energy/Infrastructure → Energy and Mining
Energy/Oil and Gas -> Energy/Oil and Gas: Energy and Mining
Engineering -> Engineering
Engineering & Construction -> Construction
Engineering Services -> Engineering Services -> Engineering
Financials -> Financials: Finance
Food -> Food -> Consumer Goods
Food and Agriculture -> Manufacturing
Food and Beverage/Spices -> Food and Beverage/Spices: Consumer Goods
Health Care -> Healthcare -> Healthcare
Healthcare Services -> Healthcare Services -> Healthcare
Healthcare Technology -> Healthcare Technology: Healthcare
Healthcare/Hospitals -> Healthcare/Hospitals -> Healthcare
Healthcare/Technology -> Healthcare/Technology: Healthcare
Lab Equipment -> Here are the lab equipment mapped t

In [51]:
sum = 0
for key,value in sector_mapping.items():
    if value not in real_salary_sectors:
        print(f"{key} -> {value}")
        sum += 1

Aerospace/Building Systems -> Building Systems -> Construction
Beverages -> Beverages -> Consumer Goods
Consumer Discretionary -> Consumer Discretionary -> Entertainment
Consumer Goods/Homecare -> Consumer Goods/Homecare → Consumer Goods
Electronics -> Electronics: Hardware and Networking
Energy -> Energy
Energy/Infrastructure -> Energy/Infrastructure: Energy and Mining
Energy/Oil and Gas -> Energy/Oil and Gas → Energy and Mining
Engineering -> Engineering: Manufacturing
Engineering & Construction -> Engineering & Construction: Engineering
Engineering Services -> Engineering Services
Financials -> Financials: Finance
Food -> Food -> Consumer Goods
Food and Agriculture -> Agriculture: Manufacturing
Food: Consumer Goods
Food and Beverage/Spices -> Food and Beverage/Spices: Consumer Goods
Health Care -> Health Care -> Healthcare
Healthcare Services -> Healthcare Services: Healthcare
Healthcare Technology -> Healthcare Technology: Healthcare
Healthcare/Hospitals -> Healthcare/Hospitals -> 

In [52]:
print(sum)

50


# First Results

After putting the sectors through the API 50 of them wwre mapped to something that were not in the list, these can be fixed manually

In [ ]:
# Manually map the remaining sectors that were not mapped by the API
# For a key manually change the value
sector_mapping["Aerospace/Building Systems"] = "Aerospace"
sector_mapping["Beverages"] = "Consumer Goods"
sector_mapping["Consumer Discretionary"] = "Consumer Goods"
sector_mapping["Consumer Goods/Homecare"] = "Consumer Goods"
sector_mapping["Electronics"] = "Hardware and Newtowrking"
sector_mapping["Engineering"] = "Engineering"
sector_mapping["Engineering & Construction"] = "Engineering"
sector_mapping["Engineering Services"] = "Engineering"
sector_mapping["Financials"] = "Finance"
sector_mapping["Food"] = "Consumer Goods"
sector_mapping["Food and Agriculture"] = "Consumer Goods"
sector_mapping["Food and Beverage/Spices"] = "Consumer Goods"
sector_mapping["Health Care"] = "Healthcare"
sector_mapping["Healthcare Services"] = "Healthcare"
sector_mapping["Healthcare Technology"] = "Healthcare"
sector_mapping["Healthcare/Hospitals"] = "Healthcare"
sector_mapping["Healthcare/Technology"] = "Healthcare"
sector_mapping["Lab Equipment"] = "Healthcare"
sector_mapping["Lighting and Technology"] = "Hardware and Newtowrking"
sector_mapping["Materials"] = "Manufacturing"
sector_mapping["Media"] = "Media and Communications"
sector_mapping["Media & Entertainment"] = "Media and Communications"
sector_mapping["Media and Entertainment"] = "Media and Communications"
sector_mapping["Professional Services"] = "Coorporate Services"
sector_mapping["Real Estate"] = "Real Estate"
sector_mapping["Real Estate/Construction"] = "Real Estate"
sector_mapping["Real Estate/Property Development"] = "Real Estate"
sector_mapping["Retail and Conglomerate"] = "Retail"
sector_mapping["Conglomerate"] = "Retail"
sector_mapping["Semiconductor"] = "Hardware and Newtowrking"
sector_mapping["Software"] = "Software and IT Services"
sector_mapping["Software and Technology"] = "Software and IT Services"
sector_mapping["Staffing"] = "Coorporate Services"
sector_mapping["Technology and Electronics"] = "Hardware and Newtowrking"
sector_mapping["Technology and Entertainment"] = "Media and Communications"
sector_mapping["Technology and Software"] = "Software and IT Services"
sector_mapping["Technology and Telecommunications"] = "Telecommunications"
sector_mapping["Technology/Software"] = "Software and IT Services"
sector_mapping["Transportation and Rail"] = "Transportation and Logistics"
sector_mapping["Transportation/Airlines"] = "Transportation and Logistics"
sector_mapping["Transportation/Airports"] = "Transportation and Logistics"
sector_mapping["Transportation/Logistics"] = "Transportation and Logistics"
sector_mapping["Transportation/Railways"] = "Transportation and Logistics"
sector_mapping["Transportation/Ridesharing"] = "Transportation and Logistics"
sector_mapping["Travel/Cruise Lines"] = "Recreation and Travel"
sector_mapping["Restaurants"] = "Recreation and Travel"
sector_mapping["Services"] = "Coorporate Services"
sector_mapping["Utilities"] = "Energy and Mining"

sum = 0
for key, value in sector_mapping.items():
    if value not in real_salary_sectors:
        print(f"{key} -> {value}")
        sum += 1

print(sum)


0


# Applying the mapping 

Now every unique sector is applied to a real salary sector. With this we will create a new collumn next to the original sector with the new sector

In [93]:
# Map the sectors in the dataframe by adding it to a new column
df_filtered_country["Real-Salary-Industry"] = df_filtered_country["Sector"].map(sector_mapping)

In [94]:
df_filtered_country.head(10)

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Sector,Industry,Real-Salary-Sector,Real-Salary-Industry
156,579575048679091,3 to 9 Years,MCA,$59K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,126334,...,USAJOBS,"Wireless Network Engineers design, implement, ...","{'Flexible Spending Accounts (FSAs), Relocatio...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Lendlease Group,Real Estate/Construction,Real Estate,Real Estate,Real Estate
270,288941837754490,4 to 15 Years,B.Tech,$57K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,35869,...,The Muse,Develop and implement urban planning strategie...,"{'Employee Referral Programs, Financial Counse...",Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",United Technologies Corporation,Aerospace/Building Systems,Aerospace and Defense,Aerospace,Aerospace
1140,2802376880223274,0 to 9 Years,B.Tech,$63K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,80163,...,SimplyHired,"Develop user experience (UX) strategies, creat...","{'Transportation Benefits, Professional Develo...",User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,International Flavors & Fragrances,Flavors & Fragrances,Chemicals,Consumer Goods,Consumer Goods
1591,2393222411935908,5 to 11 Years,MCA,$55K-$124K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,79017,...,Internships.com,Benefits Coordinators administer employee bene...,"{'Transportation Benefits, Professional Develo...",Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",KeyCorp,Financial Services,Commercial Banks,Finance,Finance
1609,2402523179167443,0 to 10 Years,MCA,$65K-$91K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,65144,...,USAJOBS,"Design outdoor spaces, parks, and landscapes w...","{'Childcare Assistance, Paid Time Off (PTO), R...",Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,Exelon,Energy,Utilities: Gas and Electric,Energy and Mining,Energy and Mining
1668,2429224065703046,4 to 10 Years,B.Com,$56K-$87K,"Washington, D.C.",USA,37.0902,-95.7129,Part-Time,49989,...,Jobs2Careers,User Experience Designers create intuitive and...,"{'Health Insurance, Retirement Plans, Paid Tim...",User-centered design principles UX/UI design t...,Design and improve user experiences (UX) by cr...,Zoetis,Healthcare,Pharmaceuticals,Manufacturing,Manufacturing
1860,1167270963119935,0 to 8 Years,M.Com,$55K-$110K,"Washington, D.C.",USA,37.0902,-95.7129,Contract,131825,...,Stack Overflow Jobs,A Front-End Developer specializes in creating ...,"{'Employee Referral Programs, Financial Counse...","Front-end web development HTML, CSS, JavaScrip...",Develop and maintain user interfaces for websi...,Southwest Airlines,Airlines,Airlines,Transportation and Logistics,Transportation and Logistics
1923,1269600252034514,5 to 13 Years,B.Com,$56K-$99K,"Washington, D.C.",USA,37.0902,-95.7129,Full-Time,127065,...,SimplyHired,Wedding Planners specialize in organizing wedd...,"{'Employee Assistance Programs (EAP), Tuition ...",Wedding planning Venue selection Catering and ...,"Specialize in wedding planning, assisting coup...",Diageo,Beverages,Beverages,Consumer Goods,Consumer Goods
2236,665688551170300,1 to 11 Years,PhD,$62K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Part-Time,21532,...,The Muse,A Residential Interior Designer plans and desi...,"{'Legal Assistance, Bonuses and Incentive Prog...",Residential interior design Space planning Col...,Plan and design interiors for residential spac...,PBF Energy,Energy,Petroleum Refining,Energy and Mining,Energy and Mining
2278,1705438618009444,2 to 13 Years,B.Tech,$55K-$126K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,40540,...,The Muse,A Fine Arts Instru

# Test for accuracy 

We will preform the same original guess with the API just changing the list that the API can access to be the real salary industry, then test the accuracy.

In [95]:
def getIndustry(prompt):
    real_salary_industry_str = ", ".join(real_salary_sectors)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                # What we are asking the API, will be given the name of company in the prompt
                "content": f"Identify the industry for the company named {prompt}. " 
                            "Respond strictly in the format: 'Industry: [best matching industry]'. "
                            f"Select the most appropriate industry from this list: {real_salary_industry_str}." 
                            "Do not include any additional information or commentary."
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    # extract sector and industry
    try:
        industry = str(response).split("Industry: ")[1].split("]")[0].strip()
    except IndexError:
        industry = "Unknown",

    return industry

In [96]:
reponse_industry = getIndustry("Apple")
print(reponse_industry)

Consumer Goods


In [114]:
test_size = 200

In [115]:
companies = df_filtered_country['Company']
predicted_industries = []
for company in companies[:test_size]:
    predicted_industry = getIndustry(company)
    predicted_industries.append(predicted_industry)
    print(f"{company} -> {predicted_industry}")

Lendlease Group -> Construction
United Technologies Corporation -> Aerospace
International Flavors & Fragrances -> Consumer Goods
KeyCorp -> Finance
Exelon -> Energy and Mining
Zoetis -> Healthcare
Southwest Airlines -> Transportation and Logistics
Diageo -> Consumer Goods
PBF Energy -> Energy and Mining
China Merchants Bank -> Finance
Live Nation Entertainment -> Entertainment
Booking Holdings -> Software and IT Services
Segro -> Real Estate
Computacenter -> Software and IT Services
Texas Instruments -> Electronics
Block -> Finance
Yum China Holdings -> Retail
Delta Air Lines -> Transportation and Logistics
Bayerische Motoren Werke AG (BMW) -> Manufacturing
Victrex -> Aerospace
V-Guard Industries -> Manufacturing
Hyatt Hotels Corporation -> Hospitality
Sinopec -> Energy and Mining
Diamondback Energy -> Energy and Mining
The Vanguard Group, Inc. -> Finance
Ovintiv -> Energy and Mining
The Vanguard Group, Inc. -> Finance
Global Partners -> Energy and Mining
Scottish Mortgage Investment 

In [ ]:
df_comparison = pd.DataFrame({
    'company': df_filtered_country['Company'][:test_size],
    'predicted_industry': predicted_industries,
    'industry': df_filtered_country['Real-Salary-Industry'][:test_size]
})

In [109]:
df_comparison['industry_correct'] = df_comparison['predicted_industry'] == df_comparison['industry']
industry_accuracy = df_comparison['industry_correct'].mean()
print(f"Industry accuracy: {industry_accuracy}")

Industry accuracy: 0.59


In [110]:
df_comparison

,company,predicted_industry,industry,industry_correct
156,Lendlease Group,Construction,Real Estate,False
270,United Technologies Corporation,Aerospace,Aerospace,True
1140,International Flavors & Fragrances,Consumer Goods,Consumer Goods,True
1591,KeyCorp,Finance,Finance,True
1609,Exelon,Energy and Mining,Energy and Mining,True
...,...,...,...,...
19985,Cardinal Health,Healthcare,Manufacturing,False
20397,Assurant,Finance,Finance,True
20609,MetLife,Finance,Finance,True
20761,U.S. Bancorp,Finance,Finance,True


# Results 

The folowing are the results:

1. Entries: 50, Accuracy: 60%, Time: 1:37 minutes
2. Entries: 100, Accuracy: 59%, Time: 2:22 minutes
3. Entriess 200, Accuracy 59%, Time: 5:03 minutes